In [1]:
import numpy as np
import itertools
import random
from statistics import stdev
import math

In [2]:
p_running = 0.9
p_failure = 1 - p_running
path = '/home/dkabe/Model_brainstorming/SAA_Analysis/'

In [3]:
provinces = [3, 6]
plants = 6
dc = 4
a5 = list(itertools.product([1, 0], repeat = provinces[0]))
a6 = list(itertools.product([1, 0], repeat = provinces[1]))
b = list(itertools.product([1, 0], repeat = dc))
mp_loc_encoded = [[0, 0, 1, 1, 2, 2],
                  [0, 1, 2, 3, 4, 5]]

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [5]:
province_disruptions = [[x,y] for x in a6 for y in b]
num_Scenarios = len(province_disruptions)
plant_failure = [[None]*plants for x in range(2**provinces[1])]
for k in range(len(plant_failure)):
    for i in range(plants):
        ind = mp_loc_encoded[1][i]
        plant_failure[k][i] = a6[k][ind]
Scen = [[x,y] for x in plant_failure for y in b]
p_scen = []
for s in range(len(province_disruptions)):
    p_i = (p_running**(np.sum(province_disruptions[s][0]) + np.sum(province_disruptions[s][1])))*(p_failure**(provinces[1] + dc - (np.sum(province_disruptions[s][0]) + np.sum(province_disruptions[s][1]))))
    p_scen.append(p_i)
    
with open(path + '/Evaluation_Set.txt', 'w') as f:
    for item in Scen:
        f.write("%s\n" % item)
f.close()
np.savetxt("p_scen.txt", p_scen)

In [54]:
N = [50, 100, 150, 200, 250, 300]
batches = 30

In [55]:
random.seed(1)
for num_scen in N:
    for batch in range(batches):
        indices = random.sample(range(len(Scen)), num_scen)
        sample_scenarios = [Scen[index] for index in indices]
        p_sample_scenarios = [p_scen[index] for index in indices]
        p_factor = 1/np.sum(p_sample_scenarios)
        p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))  
        with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w') as f:
            for item in sample_scenarios:
                f.write("%s\n" % item)
        f.close()
        np.savetxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt', p_sample_scenarios)


In [60]:
counter = 0 
for num_scen in N:
    for batch in range(batches):
        probs = np.loadtxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt')
        counter += np.sum(probs)

In [61]:
counter

180.0

In [7]:
p1_mean = 137000
p2_mean = 135000
p3_mean = 133000

p1_sd = 2650
p2_sd = 8125
p3_sd = 4000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

Products = 3
Market = 29

In [62]:
np.random.seed(1)

for num_scen in N:
    for batch in range(batches):
        demand = np.zeros((num_scen, Products, Market))
        for s in range(num_scen):
            demand[s][0] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
            demand[s][1] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
            demand[s][2] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))
        np.savetxt(path + 'Scen_demand/' + 'demand_' + str(num_scen) + '_' + str(batch) + '.txt', demand.reshape((num_scen*Products, Market)))


In [9]:
# forming evaluation set demand 
demand = np.zeros((1024, Products, Market))
for s in range(len(Scen)):
    demand[s][0] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
    demand[s][1] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
    demand[s][2] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))

np.savetxt("eval_set_demand.txt", demand.reshape((1024*3,29)))

In [66]:
# 50 scenario confidence interval
batches = 30

In [73]:
path = '/home/dkabe/Model_brainstorming/SAA_Analysis/'
scenarios = [50, 100, 150, 200, 250]
for scen in scenarios:
    objvals = np.loadtxt(path + str(scen) + '_results.txt')
    avg = (np.mean(objvals))
    print(avg)

    std = (stdev(objvals))
    lbwidth = std/math.sqrt(batches)*2.145
    print('ci on lower bound = [', round(avg-lbwidth,2), ',', round(avg+lbwidth,2), ']')


19330179.58466667
ci on lower bound = [ 18187116.96 , 20473242.21 ]
19724198.245333333
ci on lower bound = [ 18803339.39 , 20645057.1 ]
19130722.44466667
ci on lower bound = [ 18103873.7 , 20157571.19 ]
19214189.037666667
ci on lower bound = [ 18549614.86 , 19878763.21 ]
18660917.756333333
ci on lower bound = [ 17924103.15 , 19397732.36 ]


In [64]:
# Upper bound CI
scenarios = [50, 100, 150, 200, 250]
for scen in (scenarios):
    objvals = np.loadtxt("/home/dkabe/Model_brainstorming/SAA_Analysis/" + str(scen) + "_UB_results.txt")
    p_scen = np.loadtxt("/home/dkabe/Model_brainstorming/SAA_Analysis/p_scen.txt")
    avg = (np.sum(p_scen*objvals))
    std = math.sqrt(np.sum(p_scen*((objvals - avg)**2)))
    ubwidth = std/math.sqrt(1024)*1.96
    print('ci on upper bound = [', round(avg-ubwidth,2), ',', round(avg+ubwidth,2), ']')


ci on upper bound = [ 19393599.38 , 20678313.03 ]
ci on upper bound = [ 19628629.04 , 20968281.97 ]
ci on upper bound = [ 18650040.46 , 19416250.29 ]
ci on upper bound = [ 18766639.84 , 19481237.34 ]
ci on upper bound = [ 19494724.69 , 20780684.09 ]


In [3]:

import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  40


[2, 2, 3, 4, 2, 3, 2, 3, 1, 2]


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


In [6]:
pool

Process ForkPoolWorker-19:


Process ForkPoolWorker-34:


Process ForkPoolWorker-21:


Process ForkPoolWorker-29:


Process ForkPoolWorker-24:


...

Process ForkPoolWorker-17:


Process ForkPoolWorker-12:


Process ForkPoolWorker-14:


Process ForkPoolWorker-13:


Process ForkPoolWorker-11:
